#  大纲

2022/05/19- 05/22

- [X] 二分搜索：套路总结
- [X] 时间空间复杂度。通常为O(logn)
- [X] leetcode典型例题
- [X] 我的总结
- [X] 实际应用

参考链接：
- [leetcode cookbook](https://books.halfrost.com/leetcode/ChapterTwo/Binary_Search/)
- [宫水三叶-排序wiki](https://github.com/SharingSource/LogicStack-LeetCode/wiki/%E6%8E%92%E5%BA%8F)
- [dowalle/algo](https://github.com/dowalle/algo/blob/main/Algorithm/2-%E7%AE%97%E6%B3%95%E5%9F%BA%E7%A1%80/2-%E4%BA%8C%E5%88%86%E6%B3%95.md)

# 知识点


- [X] 套路总结

每次需要查找集合中满足条件的索引或元素时，都应该考虑二分查找。

如果集合是无序的，我们可以总是在应用二分查找之前先对其进行排序。

「使……最大值尽可能小」是二分搜索题目常见的问法。

- 分析二分查找的一个技巧是：不要出现 else，而是把所有情况用 else if 写清楚，这样可以清楚地展现所有细节。
- 另外提前说明一下，计算 mid 时需要防止溢出，代码中 `left + (right - left) // 2` 就和 `(left + right) // 2` 的结果相同，但是有效防止了 left 和 right 太大，直接相加导致溢出的情况。

思路：
1. 确定范围，最大值和最小值
2. 二分地去猜，并判定是否满足条件possible(guess)，**将一个可满足问题，变为可判定问题**，复杂度会下降很多的！
3. 不知道如何写更新左右下标的判定条件时，就先把满足的条件列出来；再针对不满足的情况一一讨论
4. 不知道如何计算mid值的时候，就假设窗口最小的极端情况，看mid取左还是取右，会不会出现死循环


几个特别要注意的点

- 当窗口达到最小的时候，如果存在目标id，
    - mid和left right的关系
    - left和right的关系
- 判定条件是什么？如何确定下一步该走到左边还是右边？不满足的时候，如何更新左右下标？
    - 右端点是否闭，影响退出循环的判断和下标更新的方式（不过，只是形式上的影响，不是实质上的），在计算mid值的时候也要小心处理下，注意是否会出现死循环

## 常见情况

根据判定条件和目标id之间的关系，是否需要判断左右邻居，可分为以下几种情形进行处理

### 标准情形

目标id的值 等于 判定条件即可，无需判断左右邻居
- 即满足退出循环的条件是nums[mid] == target
- 当窗口达到最小的时候，
    - 如果选择左闭右开，即left=right-1时，mid=left=right-1
    - 如果选择左闭右闭，即left=rgiht时，mid=left=rgiht
- 无法满足时，退出循环的条件
    - 如果选择左闭右开（即左下标可以取到，右下标不可以取到），left>=right时退出
    - 如果选择左闭右闭，left>right时退出  
- 如果不满足的话，左右下标的更新方式：left=mid-1，right=mid+1

In [ ]:
# 标准模板

def binarySearch(nums, target):
    """
    :type nums: List[int]
    :type target: int
    :rtype: int
    """
    if len(nums) == 0:
        return -1

    left, right = 0, len(nums) - 1 # 左闭右闭的方式
    
    while left <= right: # 退出条件是左严格大于右，
        # 当左右相等的时候，依然要和目标比较
        # 等价于：查找条件可以在不与元素的两侧进行比较的情况下确定（或使用它周围的特定元素）。
        mid = (left + right) // 2
        if nums[mid] == target:
            return mid
        elif nums[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1   

### 查找第一个与 target 相等的元素

目标id的值 等于 判定条件，且需判断**左**邻居不存在或小于目标值

- 即满足退出循环的条件是nums[mid] == target and (mid == 0 or nums[mid-1] < target),即最终的窗口里应该包含两个数[mid-1,mid]，或者[0]. 对于mid=0，这个可以一开始就判断nums[0]是否等于target来决定。
- 当窗口达到最小的时候，
    - 如果选择左闭右开，即left=right-2时，mid=left+1=right-1
    - 如果选择左闭右闭，即left=rgiht-1时，mid=left+1=rgiht
- 下面讨论nums[0] != target，无法满足时退出循环的条件
    - 如果选择左闭右开，right代表大于target值或非第一个等于target的值；left>right-2 (<=>)right-left<=1
    - 如果选择左闭右闭，right代表大于或等于target的值；right-left<=0
- 如果不满足的话，左右下标的更新方式：
    - 如果选择左闭右开: if (nums[mid] > target) or (nums[mid] == target and nums[mid-1] >= target), right=mid
    - 如果选择左闭右闭: if (nums[mid] > target) or (nums[mid] == target and nums[mid-1] >= target), right=mid - 1
    - 更新左端点的方式一样：if nums[mid] < target, left=mid


In [19]:
# 左闭右开
def BinarySearch(nums, target):
    if len(nums) < 1:
        return -1
    if nums[0] == target: # 单独处理首位与target相等的情况
        return 0
    left = 0
    right = len(nums)
    while right-left > 1: # 退出条件是窗口里只有一个数
        mid = (right + left) // 2 
        if nums[mid] > target or (nums[mid] == target and nums[mid-1]>=target):
            right = mid # right代表严格大于判定条件的位置
        elif nums[mid] < target:
            left = mid # left代表小于target的值
        else:
            return mid
    return -1

def BinarySearchV2(nums, target):
    if len(nums) < 1:
        return -1
    left = 0
    right = len(nums)
    while left < right: # 退出条件是窗口里只有一个数
        mid = (right + left) // 2 
        if nums[mid] < target:
            left = mid + 1 # left代表小于target的值
        else:
            right = mid
    if left == len(nums) or nums[left] != target:
        return -1
    return left

# 左闭右闭
def BinarySearchClosed(nums, target):
    if len(nums) < 1:
        return -1
    if nums[0] == target:
        return 0
    left = 0
    right = len(nums) - 1 # 右下标可取到
    while right - left > 0: # 退出条件是窗口里只有一个数
        mid = (right + left) // 2 + 1 # 注意这里要加1，针对right=left+1的情况，这样计算mid=left+1，否则的话，mid=left，会陷入死循环
        if nums[mid] > target or (nums[mid] == target and nums[mid-1]>=target):
            right = mid-1 # right可以取到
        elif nums[mid] < target:
            left = mid
        else:
            return mid
    return -1

In [21]:
nums = [1,1,1,3,4,5,5,5,6,6,6,7,8]
target = 8
print(BinarySearch(nums, target))
print(BinarySearchV2(nums, target))
print(BinarySearchClosed(nums, target))

12
12
12


### 查找最后一个与 target 相等的元素

目标id的值 等于 判定条件，且需判断**右邻居**不存在或大于目标值

- 即满足退出循环的条件是nums[mid] == target and (mid == len(nums)-1 or nums[mid+1] > target),即最终的窗口里应该包含两个数[mid,mid+1]，或者[len(nums)-1]. 对于mid=len(nums)-1，这个可以一开始就判断nums[-1]是否等于target来决定。
- 当窗口达到最小的时候，
    - 如果选择左闭右开，即left=right-2时，mid=left=right-2
    - 如果选择左闭右闭，即left=rgiht-1时，mid=left=rgiht-1
- 下面讨论nums[-1] != target，无法满足时退出循环的条件
    - 如果选择左闭右开，right代表**非第一个**大于target值；因为最小窗口是left=right-2，则退出条件是left>right-2 (<=>) right < left + 2 or right <= left + 1
    - 如果选择左闭右闭，right代表大于target的值；同理，right-left<=0 or right < left + 1
- 如果不满足的话，左右下标的更新方式：
    - 如果选择左闭右开: 
        - if (nums[mid] > target) right = mid + 1
        - if (nums[mid] < target) left = mid + 1
        - if (nums[mid] == target and nums[mid+1] <= target) left=mid+1
    - 如果选择左闭右闭: 
        - if (nums[mid] > target) right = mid
        - if (nums[mid] < target) left = mid + 1
        - if (nums[mid] == target and nums[mid+1] <= target), left = mid + 1


In [29]:
# 左闭右开
def BinarySearch(nums, target):
    if len(nums) < 1:
        return -1
    left = 0
    right = len(nums)
    while left < right: # 退出条件是窗口里只有一个数
        mid = (right + left) // 2
        if nums[mid] > target:
            right = mid
        else:
            left = mid + 1
    if nums[left-1] != target:
        return -1
    return left - 1

# 左闭右闭
def BinarySearchClosed(nums, target):
    if len(nums) < 1:
        return -1
    if nums[-1] == target:
        return len(nums)-1
    left = 0
    right = len(nums) - 1 # 右下标可取到
    while right - left > 0: # 退出条件是窗口里只有一个数
        mid = (right + left) // 2 # 注意这里不用加1，针对right=left+1的情况，mid=left即可
        if nums[mid] < target or (nums[mid] == target and nums[mid+1]<=target):
            left = mid + 1
        elif nums[mid] > target:
            right = mid # 这里不加1，right代表大于target的值
        else:
            return mid
    return -1

In [34]:
nums = [1,1,1,3,4,5,5,5,6,6,6,7,8]
target = 3
print(BinarySearch(nums, target))
print(BinarySearchClosed(nums, target))

3
3


### 查找第一个大于等于 target 的元素

和第一个等于target的元素整体思路一致，在判定条件上有变化

目标id的值 **大于等于** 判定条件，且需判断**左**邻居不存在或小于目标值

- 即满足退出循环的条件是nums[mid] >= target and (mid == 0 or nums[mid-1] < target),即最终的窗口里应该包含两个数[mid-1,mid]，或者[0]. 对于mid=0，这个可以一开始就判断nums[0]是否等于target来决定。
与第一个等于target的元素不同的地方在于：
- 如果不满足的话，左右下标的更新方式：
    - 如果选择左闭右开: 
        - if nums[mid] < target, left = mid
        - if nums[mid] >= target and nums[mid-1] >= target, right = mid
    - 如果选择左闭右闭: 
        - if nums[mid] < target, left = mid
        - if nums[mid] >= target and nums[mid-1] >= target, right = mid - 1

In [8]:
# 左闭右开
def BinarySearch(nums, target):
    if len(nums) < 1:
        return -1
    if nums[0] == target: # 单独处理首位与target相等的情况
        return 0
    left = 0
    right = len(nums)
    while right-left > 1: # 退出条件是窗口里只有一个数
        mid = (right + left) // 2 
        if nums[mid] < target:
            left = mid # left代表小于target的值
        elif nums[mid-1] >= target:
            right = mid 
        else:
            return mid
    return -1

def BinarySearchV2(nums, target):
    if not nums:
        return -1
    l = 0
    r = len(nums)
    while l < r:
        mid = l + (r-l)//2
        if nums[mid] >= target:
            r = mid
        else:
            l = mid + 1
    return r
    

# 左闭右闭
def BinarySearchClosed(nums, target):
    if len(nums) < 1:
        return -1
    if nums[0] == target:
        return 0
    left = 0
    right = len(nums) - 1
    while right - left > 0: 
        mid = (right + left) // 2 + 1 # 注意这里要加1，针对right=left+1的情况，这样计算mid=left+1，否则的话，mid=left，会陷入死循环
        if nums[mid] < target:
            left = mid 
        elif nums[mid-1] >= target:
            right = mid - 1
        else:
            return mid
    return -1

In [11]:
nums = [1,1,1,3,5,5,5,5,6,6,6,7,8]
target = 2
print(BinarySearch(nums, target))
print(BinarySearchV2(nums, target))
print(BinarySearchClosed(nums, target))

3
3
3


### 查找最后一个小于 target 的元素

和最后一个等于target的元素整体思路一致，在判定条件上有变化

目标id的值 **小于** 判定条件，且需判断**右**邻居不存在或大于等于目标值

- 即满足退出循环的条件是nums[mid] < target and (mid == len(nums)-1 or nums[mid+1] >= target),即最终的窗口里应该包含两个数[mid,mid+1]，或者[len(nums)-1]. 对于mid=len(nums)-1，这个可以一开始就判断nums[-1]是否小于target来决定。

与最后一个等于target的元素不同的地方在于：
- 如果不满足的话，左右下标的更新方式：
    - 如果选择左闭右开: 
        - if (nums[mid] >= target) right = mid + 1
        - if (nums[mid] < target and nums[mid+1] < target) left = mid + 1
    - 如果选择左闭右闭: 
        - if (nums[mid] >= target) right = mid
        - if (nums[mid] < target and nums[mid+1] < target) left = mid + 1

In [22]:
# 左闭右开
def BinarySearch(nums, target):
    if len(nums) < 1:
        return -1
    if nums[-1] < target: # 单独处理末位与target相等的情况
        return len(nums)-1
    left = 0
    right = len(nums)
    while right-left > 1: # 退出条件是窗口里只有一个数
        mid = (right + left) // 2 
        if nums[mid] >= target:
            right = mid + 1 # right代表严格大于target的值
        elif nums[mid+1] < target:
            left = mid + 1 # left 代表小于target的值
        else:
            return mid
    rturn -1

# 左闭右闭
def BinarySearchClosed(nums, target):
    if len(nums) < 1:
        return -1
    if nums[-1] < target:
        return len(nums)-1
    left = 0
    right = len(nums) - 1 
    while right - left > 0:
        mid = (right + left) // 2 # 注意这里不用加1，针对right=left+1的情况，mid=left即可
        if nums[mid] >= target:
            right = mid # 这里不加1，right代表大于等于target的值
        elif nums[mid+1] < target:
            left = mid + 1
        else:
            return mid
    return -1

In [23]:
nums = [1,1,1,3,5,5,5,5,6,6,6,7,8]
target = 4
print(BinarySearch(nums, target))
print(BinarySearchClosed(nums, target))

3
3


## 二分搜索问题的泛化

应用：在具体的算法问题中，常用到的是「搜索左侧边界」和「搜索右侧边界」这两种场景，很少有让你单独「搜索一个元素」。

来自[labuladong](https://labuladong.github.io/algo/2/18/28/)

搜索左边界和右边界的模板在下面，两个算法过程的区别是：
- 返回值：搜索左边界，返回left；搜索右边界，返回left-1
- 当找到target时，搜索左边界，把right赋值为mid；搜索右边界，把left赋值为mid+1


首先，你要从题目中抽象出一个自变量 `x`，一个关于 `x` 的函数 `f(x)`，以及一个目标值 target。同时，x, f(x), target 还要满足以下条件：

1. `f(x)` 必须是在 `x` 上的单调函数（单调增单调减都可以）。

2. 题目是让你计算满足约束条件 `f(x) == target` 时的 `x` 的值。

In [ ]:
# 搜索左边界
def left_bound(nums, target):
    if not nums:
        return -1
    left = 0
    right = len(nums)
    
    while left < right:
        mid = left + (right - left) // 2
        if nums[mid] == target: 
            # 当找到 target 时，收缩右侧边界
            right = mid
        elif nums[mid] < target:
            left = mid + 1
        else # if nums[mid] > target:
            right = mid
    return left

# 搜索右边界
def right_bound(nums, target):
    if not nums:
        return -1
    left = 0
    right = len(nums)
    
    while left < right:
        mid = left + (right - left) // 2
        if nums[mid] == target: 
            # 当找到 target 时，收缩左侧边界
            left = mid + 1
        elif nums[mid] < target:
            left = mid + 1
        else # if nums[mid] > target:
            right = mid
    return left - 1

# 泛化的二分搜索问题的模板


# 函数 f 是关于自变量 x 的单调函数
def f(x):
    # ...
    

# 主函数，在 f(x) == target 的约束下求 x 的最值
def solution(nums, target):
    if not nums:
        return -1
    # 问自己：自变量 x 的最小值是多少？
    left = ...
    # 问自己：自变量 x 的最大值是多少？
    right = ... + 1 # 这里取开区间
    
    while left < right:
        mid = left + (right - left) // 2
        if f(mid) == target: 
            # 问自己：题目是求左边界还是右边界？
            # ...
        elif f(mid) < target:
            # 问自己：怎么让 f(x) 大一点？
            # ...
        else: # if (f(mid) > target) {
            # 问自己：怎么让 f(x) 小一点？
            # ...
    return left # or left - 1


# Leetcode例题


在基本有序的数组中用二分搜索。经典解法可以解，变种写法也可以写，常见的题型，在山峰数组中找山峰，在旋转有序数组中找分界点。
主要难点在于设计判定条件，如何选择左边还是右边？

- [(medium)33搜索旋转排序数组](https://leetcode.cn/problems/search-in-rotated-sorted-array/)。思路：整体思路和排序数组一样，与中间数做比较，根据大小决定下一个窗口。不同之处在于：决定下一个窗口要分类讨论。
- [(medium)81搜索旋转排序数组II](https://leetcode.cn/problems/search-in-rotated-sorted-array-ii/)。思路：大体上同33，但是因为有重复，所以会出现左右两端相等的情况，此时必须逐个扫描。因此最坏的情况下，时间复杂度是线性的，不能达到log。
- [(medium)153寻找旋转排序数组中的最小值](https://leetcode.cn/problems/find-minimum-in-rotated-sorted-array/)思路：大体上同33。
- [(hard)154寻找旋转排序数组中的最小值II](https://leetcode.cn/problems/find-minimum-in-rotated-sorted-array-ii/)。思路：结合153和81，额外处理两边相等的情况
- [(medium)162寻找峰值](https://leetcode.cn/problems/find-peak-element/)。思路：二分查找，关键在于下一个left，right如何选择。发现：始终选择大于边界一端进行二分，可以确保选择的区间一定存在峰值，并随着二分过程不断逼近峰值位置。
- [(medium)540有序数组中的单一元素](https://leetcode.cn/problems/single-element-in-a-sorted-array/)。思路：如何选择左边还是右边？看偶数下标和下一个奇数下标是否相等。如果每个数都恰好出现两次，必然有偶数下标的数等于下一个奇数下标的数。相等的话，走右边，left=mid + 1；不相等的话，走左边，right = mid
- [(easy)852山脉数组的峰顶索引](https://leetcode.cn/problems/peak-index-in-a-mountain-array/)/[(easy)911剑指offer69山峰数组的顶部](https://leetcode.cn/problems/B1IidL/) 思路：如何选择左边还是右边？如果中间值比左边邻居大，比右边邻居大，则找到；否则，如果中间值比左边邻居小，则知道山峰一定出现在左边；否则，如果中间值比右边邻居小，则知道山峰一定出现在右边。
- 🌟🌟[(hard)4寻找两个正序数组的中位数](https://leetcode.cn/problems/median-of-two-sorted-arrays/)
    - 思路一：寻找两个数组的第k个最小值，二分法，每次可以排除k/2个值。
    - 思路二：常规二分，难点在于设计判定条件，如何选择左边还是右边。以后有时间来实现一下。有很多很多的细节需要处理



max-min 最大值最小化问题。求在最小满足条件的情况下的最大值。整体思路是：把求值问题，转换为可判定问题。在有限的范围里猜结果，判定结果能否被满足。

- 🌟[(medium)287寻找重复数](https://leetcode.cn/problems/find-the-duplicate-number/)。这道题有多种解法，题解里的快慢指针和二进制方法，看起来很巧妙。以后学习一下。这里实现二分法。猜测重复数，然后判断该数是否满足条件，进行左右区间的选择。
- [(hard)410分割数组的最大值](https://leetcode.cn/problems/split-array-largest-sum/)。思路：转为可判定问题，使用二分法来猜测目标值。如果我们猜测的值，可以通过划分m个区间达到，则最小值应该不大于猜测值，选择左边；否则选择右边。如何判定能否通过划分m个区间达到呢？贪心法：令每段和都不大于目标值，看看最少要使用几段。如果使用的段数超过m，则证明目标值太小了无法达到。否则，真实的最小值要小于等于目标值。
- [(medium)875爱吃香蕉的珂珂](https://leetcode.cn/problems/koko-eating-bananas/)。思路：同410，先确定边界，再判定猜测值是否满足，以及如何选择左边还是右边。关键在于设计如何判定。
- [(medium)1011在D天内送达包裹的能力](https://leetcode.cn/problems/capacity-to-ship-packages-within-d-days/)。思路：和410一模一样
- [(medium)1283使结果不超过阈值的最小除数](https://leetcode.cn/problems/find-the-smallest-divisor-given-a-threshold/)。思路：同上。注意一个细节，除数必须大于等于1，即不能为0
- 🌟[(hard)668乘法表中第k小的数](https://leetcode.cn/problems/kth-smallest-number-in-multiplication-table/)。思路：给出一个数字，可以计算出在乘法表里有多少数字小于等于该数。我们要找的就是**第一个**满足有k个小于等于这个数字的数。套路：二分猜值+判定。


- [(medium)34在排序数组中查找元素的第一个和最后一个位置](https://leetcode.cn/problems/find-first-and-last-position-of-element-in-sorted-array/)。思路：基本上套用标准模板。不同点在于：当中值等于目标值的时候，要左右移动下标，找到起始和终止位置。
- [(easy)35搜索插入位置](https://leetcode.cn/problems/search-insert-position/)。思路：重点在弄清这个位置的实际含义：第一个大于或等于目标值的位置。有一个特殊情况：目标值比所有值都大的时候，返回数组长度。
- [(easy)367有效的完全平方数](https://leetcode.cn/problems/valid-perfect-square/)。基础题
- [(easy)704二分查找](https://leetcode.cn/problems/binary-search/)。基础题：标准模板
- [(easy)744寻找比目标字母大的最小字母](https://leetcode.cn/problems/find-smallest-letter-greater-than-target/)。基础题：注意处理循环的情况，即最后一个字母也比目标小，则返回第一个字母。
- [(medium)911在线选举](https://leetcode.cn/problems/online-election/)。思路：在时间点上二分查找，找到最后一个小于或等于查询时刻的时间点，再返回该时刻的得票最高者的记录。这个记录，可以在初始化的时候，对每一个时间点构造得到。
- [(medium)1818绝对差值和](https://leetcode.cn/problems/minimum-absolute-sum-difference/)
    - 思路一：使用在限制条件下找最值的套路：先圈定范围，猜可能的最小和，然后判定能否达到。时间复杂度比较高，虽然外层的二分查找是logn的，但是内层的判定是n2的，总体是n2logn的
    - 思路二：对于每一个i，能确定它最大的提升空间：一个nums1[i]，对应两个diff：
        - 当前的diff: |nums1[i] - nums2[i]|，
        - 与对应nums2最接近的nums1里的数的的差值: |nums1[j] - nums2[i]|。寻找j的话，可以先对nums1进行排序，然后使用二分法在nums1里寻找最接近nums2[i]的数。
        这两个之间的差值，就是i可带来的提升空间。返回最大的提升空间。时间复杂度是O(nlogn)的。排序是O(nlogn)的，对每一个i遍历是O(n)的，遍历中寻找最小的diff是O(logn)的，综合来看是O(nlogn)的
- [(easy)1984学生分数的最小差值](https://leetcode.cn/problems/minimum-difference-between-highest-and-lowest-of-k-scores/)
    - 思路一：滑动窗口：先排序，再比较是否小于当前最小值。时间复杂度是O(nlogn)的
- [(easy)剑指offer53 在排序数组中查找数字I](https://leetcode.cn/problems/zai-pai-xu-shu-zu-zhong-cha-zhao-shu-zi-lcof/)。思路：基础题。类似于34.


In [ ]:
# 33 二分查找，分类讨论
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        left = 0
        right = len(nums) - 1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] == target:
                return mid
            if nums[left] <= nums[right]: # 和有序的情况一致
                if target < nums[left] or  target > nums[right]: 
                    return -1
                if target == nums[left]:
                    return left
                if target == nums[right]:
                    return right
                if nums[mid] < target: # 比中间值小，则放到更小的一边
                    left = mid + 1
                else:
                    right = mid - 1
            # 其余的情况都表明，该数组经历了一次逆序。
            elif nums[mid] < target:  # 目标比中间值大，下一个要寻找“较大”的那一半
                if nums[mid] <= nums[right] and nums[right] < target: # 中间值比右边值小，代表右边有序；右边值比目标值小，代表要去到左边。
                    right = mid - 1
                else: 
                    left = mid + 1
            else: # 目标比中间值小，下一个要寻找“较小”的那一半
                if nums[mid] > nums[right] and target <= nums[right]: # 中间值比右边值大，代表右边逆序；当目标值比右边值小的时候，去到右边。
                    left = mid + 1
                else:
                    right = mid - 1
        return -1

In [ ]:
# 81 分类讨论
class Solution:
    def search(self, nums: List[int], target: int) -> bool:
        left = 0
        right = len(nums) - 1
        while left <= right:
            if nums[left] == nums[right]: # 当左右两端相等的时候，无法判断出应该走向哪边。需要先把相等的这些数字“去掉”才行。
                if nums[left] == target:
                    return True
                while left < right and nums[left] == nums[right]: # 如果持续相等，就持续移动左右边界。
                    left += 1
                    if nums[left] != nums[right]:
                        break
                    right -= 1
                if left > right:
                    return False
                
            mid = (left + right) // 2
            if nums[mid] == target or nums[left] == target or nums[right] == target:
                return True
            # 以下所有：左右都不相等
            if nums[left] < nums[right]: # 和有序的情况一致
                if target < nums[left] or  target > nums[right]: 
                    return False
                if nums[mid] < target: # 比中间值小，则放到更小的一边
                    left = mid + 1
                else:
                    right = mid - 1
            # 其余的情况都表明，该数组经历了一次逆序。
            elif nums[mid] < target:  # 目标比中间值大，下一个要寻找“较大”的那一半
                if nums[mid] <= nums[right] and nums[right] < target: # 中间值比右边值小，代表右边有序；右边值比目标值小，代表要去到左边。
                    right = mid - 1
                else: 
                    left = mid + 1
            else: # 目标比中间值小，下一个要寻找“较小”的那一半
                if nums[mid] > nums[right] and target < nums[right]: # 中间值比右边值大，代表右边逆序；当目标值比右边值小的时候，去到右边。
                    left = mid + 1
                else:
                    right = mid - 1
        return False

In [ ]:
# 153 二分查找，每次走向最小值可能存在的区间
class Solution:
    def findMin(self, nums: List[int]) -> int:
        left, right = 0, len(nums) - 1
        while left <= right:
            if nums[left] <= nums[right]:
                return nums[left]
            mid = (left + right) // 2
            if nums[mid] < nums[left]:
                right = mid
            else:
                left = mid + 1

In [ ]:
# 154 结合81和153，需要处理左右两边相等的情况
class Solution:
    def findMin(self, nums: List[int]) -> int:
        left = 0
        right = len(nums) - 1
        while left <= right:
            while left < right and nums[left] == nums[right]:
                left += 1
                if nums[left] != nums[right]:
                    break
                right -=1
            if left >= right:
                return nums[right]
            mid = (left + right) // 2
            if nums[left] < nums[right]: # 顺序的情况
                return nums[left]
            else: # 逆序的情况
                if nums[mid] >= nums[left]:
                    left = mid + 1
                else:
                    right = mid


In [ ]:
# 162 两种写法的不同之处在于：第一种写法有early stop。
# 第二种写法简化了很多，但是有相当多的细节要处理，比如left<right，而非left<=right
# 更新下标时right = middle，而不是right = middle - 1；能保证middle+1一定能取到。
# 简化写的方式是，把要分类讨论的情况，都通过细节处理来省略掉。

class Solution:
    def findPeakElement(self, nums: List[int]) -> int:
        left = 0
        right = len(nums) - 1
        while left <= right:
            larger_left = False
            larger_right = False
            mid = (left + right) // 2
            if mid == 0 or nums[mid] > nums[mid - 1]:
                larger_left = True
            if mid == len(nums) - 1 or nums[mid] > nums[mid + 1]:
                larger_right = True
            if larger_left and larger_right:
                return mid # early stop
            if larger_left:
                left = mid + 1
            else:
                right = mid - 1
        return -1


class Solution:
    def findPeakElement(self, nums: List[int]) -> int:
        left = 0
        right = len(nums) - 1
        while left < right:
            middle = (left + right) // 2
            if nums[middle] < nums[middle + 1]: # 始终选择大于边界一端进行二分，可以确保选择的区间一定存在峰值
                left = middle + 1
            else:
                right = middle
        return left


In [ ]:
# 540: 重点在于如何确定该去到左边还是右边，以及如何更新下标
class Solution:
    def singleNonDuplicate(self, nums: List[int]) -> int:
        left = 0
        right = len(nums) - 1
        while left < right: 
            even_mid = (left + right) // 2 # 偶数下标
            if even_mid % 2 == 1:
                even_mid -= 1 # 如果中间值为奇数，我们知道一个奇数下标前必然有一个偶数下标，因为最小的下标是0，是偶数。所以-=1不会越界。
            odd_mid = even_mid + 1
            if nums[even_mid] == nums[odd_mid]: # 如果相等，知道只出现1次的数在右边。
                left = odd_mid + 1 # 从奇数下标的下一个开始
            else:
                right = even_mid # 如果不相等，知道只出现一次的数在左边。
        return nums[left]
    
# 化简版，去掉了变量odd_mid
class Solution:
    def singleNonDuplicate(self, nums: List[int]) -> int:
        left = 0
        right = len(nums) - 1
        while left < right:
            mid = (left + right) // 2
            if mid % 2 == 1:
                mid -= 1
            if nums[mid] == nums[mid + 1]:
                left = mid + 2
            else:
                right = mid
        return nums[left]

In [ ]:
# 852 / 剑指offer II069
class Solution:
    def peakIndexInMountainArray(self, arr: List[int]) -> int:
        left, right = 0, len(arr) - 1 # the diff >= 2
        while left <= right - 2: # 始终保持窗口里有至少三个数
            mid = (left + right) // 2 # mid严格大于left，严格小于right，所以mid-1，mid+1，一定在窗口内
            if arr[mid] > arr[mid - 1] and arr[mid] > arr[mid + 1]: # 找到了山峰。
                return mid
            if arr[mid] < arr[mid - 1]: # 比左边邻居小，山峰一定在左边
                right = mid
            else: # 比右边邻居小，山峰一定在右边
                left = mid
        return -1

In [ ]:
# 4 思路一：寻找第k个最小值

class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        # change  this to find the [m+n]/2 smallest number
        len1 = len(nums1)
        len2 = len(nums2)
        k = (len1 + len2 + 1) // 2
        module = (len1 + len2 + 1) % 2
        k_smallest, nums1_new, nums2_new = self.findKthItem(nums1, nums2, k)
        if module == 1:
            kplus1_smallest, _, _ = self.findKthItem(nums1_new, nums2_new, 1)
            return (k_smallest + kplus1_smallest) / 2
        return k_smallest

    def findKthItem(self, nums1, nums2, k):
        # print(nums1, nums2, k)
        if len(nums1) == 0:
            return nums2[k-1], [], nums2[k:]
        if len(nums2) == 0:
            return nums1[k-1], nums1[k:], []
        if k == 1:
            if nums1[0] < nums2[0]:
                return nums1[0], nums1[1:], nums2
            else:
                return nums2[0], nums1, nums2[1:]
        half_k = k // 2
        nums1_id = min(len(nums1), half_k) - 1
        nums2_id = min(len(nums2), half_k) - 1
        if nums1[nums1_id] < nums2[nums2_id]:
            return self.findKthItem(nums1[(nums1_id + 1): ], nums2,  k-nums1_id-1)
        if nums1[nums1_id] == nums2[nums2_id]:
            left_k = k - nums1_id - nums2_id - 2
            if left_k == 0:
                return nums1[nums1_id], nums1[nums1_id+1:], nums2[nums2_id + 1:]
            else:
                return self.findKthItem(nums1[nums1_id+1:], nums2[nums2_id + 1:], left_k)
        return self.findKthItem(nums1, nums2[nums2_id + 1:],  k-nums2_id-1)

In [ ]:
# 287 二分法
class Solution:
    def findDuplicate(self, nums: List[int]) -> int:
        left, right = 0, len(nums) - 1
        while left < right:
            mid = (left + right) // 2 + 1
            cnt = 0
            for num in nums:
                if num < mid:
                    cnt += 1
            if cnt < mid:
                left = mid
            else:
                right = mid - 1
        return left

In [ ]:
# 410 转为可判定问题，使用二分法来猜测目标值
class Solution:
    def achievable(self, target, nums, m): # 判定方式
        cnt = 1
        cur_sum = nums[0]
        for num in nums[1:]:
            cur_sum += num
            if cur_sum > target:
                cur_sum = num
                cnt += 1
        return cnt <= m

    def splitArray(self, nums: List[int], m: int) -> int:
        ttl_sum = sum(nums)
        left = ttl_sum // m
        if ttl_sum % m > 0:
            left += 1
        left = max(left, max(nums)) # 寻找下界
        right = ttl_sum - sum(nums[:m-1]) # 寻找上界
        while left < right:
            mid = (left + right) // 2
            if self.achievable(mid, nums, m): # 判定是否成立，成立选择左边，不成立选择右边
                right = mid
            else:
                left = mid + 1
        return left


In [ ]:
# 875 同410
class Solution:
    def CanFinish(self, mid, piles, h):
        cnt = 0
        for pile in piles:
            if pile <= mid:
                cnt += 1
            else:
                cur_cnt = pile // mid
                if pile % mid > 0:
                    cur_cnt += 1
                cnt += cur_cnt
        return cnt <= h
        
    def minEatingSpeed(self, piles: List[int], h: int) -> int:
        ttl = sum(piles) 
        left = ttl // h
        if ttl % h > 0:
            left += 1
        right = max(piles)
        while left < right:
            mid = (left + right) // 2
            if self.CanFinish(mid, piles, h):
                right = mid
            else:
                left = mid + 1
        return left


In [ ]:
# 1011 同410
class Solution:
    def CanShip(self, capacity, weights, days):
        actual_days = 1
        cur_sum = weights[0]
        for weight in weights[1:]:
            cur_sum += weight
            if cur_sum > capacity:
                cur_sum = weight
                actual_days += 1
        return actual_days <= days

    def shipWithinDays(self, weights: List[int], days: int) -> int:
        ttl_weights = sum(weights) 
        left = ttl_weights // days
        if ttl_weights % days > 0:
            left += 1
        left = max(left, max(weights))
        right = ttl_weights
        while left < right:
            mid = (left + right) // 2
            if self.CanShip(mid, weights, days):
                right = mid
            else:
                left = mid + 1
        return left

In [ ]:
# 1283
class Solution:
    def UnderThreshold(self, divisor, nums, threshold):
        actual_sum = 0
        for num in nums:
            actual_sum += (num // divisor)
            if num % divisor > 0:
                actual_sum += 1
        return actual_sum <= threshold
                
    def smallestDivisor(self, nums: List[int], threshold: int) -> int:
        left = max(sum(nums) // threshold, 1) # 这里注意：除数要大于等于1.
        right = max(nums)
        while left < right:
            mid = (left + right) // 2
            if self.UnderThreshold(mid, nums, threshold):
                right = mid
            else:
                left = mid + 1
        return left

In [ ]:
# 34
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        left = 0
        right = len(nums) - 1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] < target:
                left = mid + 1
            elif nums[mid] > target:
                right = mid - 1
            else:
                start_id = mid
                while start_id >= 0 and nums[start_id] == target:
                    start_id -= 1
                start_id += 1
                end_id = mid
                while end_id <= len(nums) - 1 and nums[end_id] == target:
                    end_id += 1
                end_id -= 1
                return [start_id, end_id]

        return [-1, -1]

In [ ]:
# 35
class Solution:
    def searchInsert(self, nums: List[int], target: int) -> int:
        # find the id of the first element equal to or larger than the target
        left = 0
        right = len(nums) - 1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] < target:
                left = mid + 1
            elif nums[mid] == target:
                return mid
            else:
                if left == right:
                    break
                right = mid
        return left

In [ ]:
# 367 
class Solution:
    def isPerfectSquare(self, num: int) -> bool:
        left = 1
        right = num
        while left <= right:
            mid = (left + right) // 2
            actual_square = mid ** 2
            if actual_square > num:
                right = mid - 1
            elif actual_square == num:
                return True
            else:
                left = mid + 1
        return False

In [ ]:
# 744
class Solution:
    def nextGreatestLetter(self, letters: List[str], target: str) -> str:
        if letters[0] > target or letters[-1] <= target: # 注意处理循环的情况
            return letters[0]
        left = 0
        right = len(letters) - 1
        while left < right:
            mid = (left + right) // 2
            if letters[mid] <= target:
                left = mid + 1
            else:
                right = mid
        return letters[left]

In [ ]:
# 911 初始化时构建一个max vote person记录，对应于每个时间点。在时间点上二分查找，返回相应时间点的记录
class TopVotedCandidate:

    def __init__(self, persons: List[int], times: List[int]):
        self.persons = persons
        self.times = times
        self.cummu = {} # 累计到目前时刻，各人的得票数
        self.max_person = [] # 初始化时构建一个max vote person记录
        self.max_vote = 0
        self.ttl_len = len(persons)
        for i in range(self.ttl_len):
            person = persons[i]
            self.cummu[person] = self.cummu.get(person, 0) + 1
            if self.cummu[person] >= self.max_vote: # 平票的话，默认最后得票的人领先。
                self.max_vote = self.cummu[person]
                self.max_person.append(person)
            else:
                self.max_person.append(self.max_person[-1])

    def q(self, t: int) -> int:
        # first, find the id of the last time smaller or equal to the t
        # use the binary search on self.times
        left = 0
        right = self.ttl_len - 1
        if self.times[right] <= t:
            return self.max_person[right]
        if self.ttl_len == 1:
            return self.max_person[0]
        while left < right:
            mid = (left + right) // 2 + 1
            if self.times[mid] < t:
                left = mid
            elif self.times[mid] == t:
                return self.max_person[mid]
            else:
                right = mid - 1
        # compare and return the result
        return self.max_person[left]



# Your TopVotedCandidate object will be instantiated and called as such:
# obj = TopVotedCandidate(persons, times)
# param_1 = obj.q(t)

In [ ]:
# 1818 思路一：使用在限制条件下找最值的套路：先猜，然后判定
class Solution:
    def CanGet(self, reduced_diff, nums1, nums2, diff):
        for i in range(len(diff)): # O(n)
            if diff[i] < reduced_diff:
                continue
            ideal_diff = diff[i] - reduced_diff
            for target_num in range(nums2[i] - ideal_diff, nums2[i] + ideal_diff + 1):  # O(n)
                if target_num in nums1:
                    return True
        return False

    def minAbsoluteSumDiff(self, nums1: List[int], nums2: List[int]) -> int:
        diff = [abs(nums1[i] - nums2[i]) for i in range(len(nums1))]
        max_sum = sum(diff)
        right = max_sum
        left = right - max(diff)
        while left < right: # logn的复杂度
            mid = (left + right) // 2
            if self.CanGet(max_sum-mid, nums1, nums2, diff):
                right = mid
            else:
                left = mid + 1
        return left % (10**9 + 7)

In [ ]:
# 1818 思路二：最大可提升的diff

class Solution:
    def findMostClosed(self, nums, target):
        # in nums find the one with the smallest difference from target and return the diff
        left = 0
        right = len(nums) - 1
        min_diff = abs(nums[0][0] - target)
        while left <= right:
            mid = (left + right) // 2
            if nums[mid][0] == target:
                return 0
            if nums[mid][0] < target:
                if target - nums[mid][0] < min_diff:
                    min_diff = target - nums[mid][0]
                left = mid + 1
            else:
                if nums[mid][0] - target < min_diff:
                    min_diff = nums[mid][0] - target
                right = mid - 1
        return min_diff
            
    def minAbsoluteSumDiff(self, nums1: List[int], nums2: List[int]) -> int:
        to_sort_nums1 = [(num, i) for i, num in enumerate(nums1)]
        to_sort_nums1.sort()
        diff = [abs(item[0] - nums2[item[1]]) for item in to_sort_nums1]
        max_sum = sum(diff)
        max_reduced = 0
        for i, item in enumerate(to_sort_nums1):
            if diff[i] <= max_reduced:
                continue # 这个early continue，把时间开销从1388ms提升到了300ms
            closed_diff = self.findMostClosed(to_sort_nums1, nums2[item[1]])
            if diff[i] - closed_diff > max_reduced:
                max_reduced = diff[i] - closed_diff
        return (max_sum - max_reduced) % (10**9 + 7)

In [ ]:
# 1984 思路一：滑动窗口：排序，然后比较
class Solution:
    def minimumDifference(self, nums: List[int], k: int) -> int:
        if k == 1:
            return 0
        nums.sort()
        min_diff = nums[k-1] - nums[0]
        for i in range(1, len(nums) - k + 1):
            if nums[i+k-1] - nums[i] < min_diff:
                min_diff = nums[i + k - 1] - nums[i]
        return min_diff

In [ ]:
# 剑指offer 53
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        left = 0
        right = len(nums) - 1
        start_id = -1
        end_id = -1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] == target:
                start_id = end_id = mid
                while start_id >= 0 and nums[start_id] == target:
                    start_id -= 1
                start_id += 1
                while end_id <= len(nums) - 1 and nums[end_id] == target:
                    end_id += 1
                end_id -= 1
                return end_id - start_id + 1
            elif nums[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return 0

In [ ]:
# 668
class Solution:
    def CountNumber(self, goal, m, n):
        cnt = 0
        for i in range(min(m, goal)):
            cnt += min(n, goal // (i + 1))
        return cnt 
    
    def findKthNumber(self, m, n, k):
        left = 1
        right = m * n 
        while left < right:
            mid = (left + right) // 2
            cnt = self.CountNumber(mid, m, n)
            if cnt >= k:
                right = mid
            else:
                left = mid + 1
        return right


# 我的总结

- 对于在旋转后的顺序数组中进行二二分法，需要注意的一点是：要处理二分后，区间是顺序的情况。套路就是详细地分类讨论。参考例题33，81，153
- 对于在限制条件下找最值问题：套路：圈定范围后去猜数，判定是否满足条件，再一步步缩小范围。